In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
import re
from pandas import DataFrame, Series

In [20]:
file_path = '/Users/kaiyanpeng/Documents/Jan/Data/bk/homes_final.csv'
temp = pd.read_csv(file_path)

In [21]:
# 去掉缺失值>0.1的列 
temp = temp.loc[:, temp.count() * 1.0 / len(temp) > 0.1]

In [22]:
# 去掉特定列
to_drop = ['Z_estimate', 'Last sold',
           'Green Energy','Parcel #', 'Last sale price/sqft', 'Country', 'CountyOrParish','num']
temp = temp.drop(to_drop, axis=1)

In [23]:
temp.shape

(817179, 61)

In [24]:
# 去掉重复行
# temp.drop_duplicates(['sold_date', 'price', 'address', 'detail_url'], inplace=True)
temp.drop_duplicates(['sold_date', 'price', 'address'], inplace=True)
temp.shape

(230049, 61)

# 处理时间、价格

In [25]:
pattern = '\d{1,2}\/\d{1,2}\/\d{1,2}'

def cleantime(x):
    if re.match(pattern, x):
        return datetime.strptime(x, '%m/%d/%Y')
    else:
        return np.NAN

temp['sold_date'] = temp['sold_date'].map(cleantime)
temp = temp[temp['sold_date'].notnull()]
temp = temp[temp.price.notnull()]

def cleanprice(x):
    c = filter(str.isdigit, x.strip('$'))
    if c == '':
        return np.nan
    else:
        return int(c)
temp.price = temp['price'].map(cleanprice)
temp =  temp[ temp['price'].notnull()]

temp = temp.sort_values(by = 'sold_date')
temp.index = np.arange(len(temp)) + 1
temp.shape

(226328, 61)

# 处理address

In [26]:
def clean_address(x):
    c =x.split(', ')
    cc = c[2].split(' ')[1]
    return Series([c[1], cc], index=['city', 'zipcode'])

address = temp['address'].map(clean_address)
temp['city'] = address.map(lambda x: x['city'])
temp['zipcode'] = address.map(lambda x: x['zipcode'])

In [27]:
temp.address[temp.zipcode == '']

7683      175 Compass Irvine Ca # 92618, Irvine, CA 
220662                    7038 Hilo Ave, Tahoma, CA 
221940               468-5 Lambert Ln, Standish, CA 
222051                  11555 Kyburz Dr, Kyburz, CA 
222722                 105 Hemlock St, Westwood, CA 
Name: address, dtype: object

In [28]:
"""处理zipcode中的缺失值
175 Compass Irvine Ca # 92618, Irvine, CA :92618
'7038 Hilo Ave, Tahoma, CA ' 96142
'468-5 Lambert Ln, Standish, CA ' 96128
'11555 Kyburz Dr, Kyburz, CA ' 95720
'105 Hemlock St, Westwood, CA ' 96137"""
temp.zipcode[7683] = 92618
temp.zipcode[220662] = 96142
temp.zipcode[221940] = 96128
temp.zipcode[222051] = 95720
temp.zipcode[222722] = 96137

/Users/kaiyanpeng/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/kaiyanpeng/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kaiyanpeng/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/kaiyanpeng/Library/Python/2.7/lib/python/site

In [29]:
temp.to_csv('1_all_1.csv')

In [18]:
temp.shape

(226729, 63)